Hey, here is my entry for the qosf task number 3... 

I picked this task since it was totally different to anything that I had done before. It turend out in the end that the quantum mechanics was actually the simple part, however I am pleased to have finished since I have learned a load about regex! The code should work, assuming the example .qasm file is formatted broadly like the ones that I have provided in the example section. It fully supports any number of custom gates, but does not surrport the SWAP and CSWAP gates since these dont seem native in qasm (could be using out of date doccumentation?) but the user could define these as custom gates anyway. One thing i would like to extend this code to is for loops and function interpretation, but i thought that i would keep it simple for now. Its proabbly not the most beautifyl solution, but im glad to have something that works. 

In [ ]:
import re
from qiskit import *

In [ ]:
# fle_str="qasmexample.qasm"
# fle_str="qasmexample_2.qasm"
fle_str="qasmexample_3.qasm"

#example 3 is coolest...

with open (fle_str, "r") as myfile:
    #read in data
    data=myfile.read()
#print out for me to read
print(data)

In [ ]:
#read in data but line by line - use this one, top is for aethstetics
with open (fle_str, "r") as myfile:
    data=myfile.readlines()
print(data)

In [ ]:
gates_sheet=("x", "y", "z", "rx", "ry", "rz", "h", "s", "sdg", "t", "tdg", "cx", "ccx", "SWAP", "CSWAP")
gates_qasm=("x", "y", "z", "rx" ,"ry" ,"x", "h", "s", "sdg", "t", "tdg", "cx c,t" , "ccx a, b, c", "SWAP", "CSWAP")
gates_qiskit=("x", "y", "z", "rx", "ry", "rz", "h", "s", "sdg", "t", "tdg", "cx", "ccx", "SWAP", "CSWAP")

In [ ]:
gates_dict_qasm_to_IBM={
    gates_qasm:gates_qasm,
}

In [ ]:
#regex sorting of the file strings - initial string for quantum, classical registers
c_names=[]
c_dims=[]
c_idx=[]

q_names=[]
q_dims=[]
q_idx=[]

custom_gates=[]
custom_idx=[]


for idx,line in enumerate(data):
    m = re.search('bit', line)
    if m:
        c_idx.append(idx)
        c_name = re.search('bit ([^[]*)', line)
        c_names.append(c_name.group(1))
        
        c_dim=re.search('\[(.*)\]', line)
        c_dims.append(c_dim.group(1))

for idx,line in enumerate(data):
    m = re.search('qubit ([^;]*)', line)
    if m:
        q_idx.append(idx)
        q_name = re.search('qubit ([^[]*)', line)
        q_names.append(q_name.group(1))
        
        q_dim=re.search('\[(.*)\]', line)
        q_dims.append(q_dim.group(1))
        
#finds the last custom gate and new search pattern from there, since it will all just be lines of the form

# gate quibits acted on

#as per the arxiv on qasm 3.0 ...https://arxiv.org/pdf/1707.03429.pdf

#going to assume these are index formatted correctly for now...


for idx,line in enumerate(data):
    for gate in gates_sheet:       
        m = re.search('}', line)
        if m:
#             print(line, gate)
            custom_idx.append(idx)

#where does the circuit bit start?

if custom_idx:
    custom_idx=max(custom_idx)
else:
    for idx,line in enumerate(data):
        for gate in gates_qasm:
            m=re.search(f'{gate}')
            if m:
                custom_idx.append(idx)
        
    custom_idx=min(custom_idx)

for idx,line in enumerate(data):
    m = re.search('gate', line)
    if m:
        n=re.search('gate\\W+(\\w+)', line)
        if (n!=None):
            custom_gates.append(n.group(1))
            

  

In [ ]:
#custom gate - handling the targets and proper placement. handled seprate to the native gates


custom_gates_begin=[]
custom_gates_end=[]
custom_gates_instructions_list=[]
custom_gates_trgts_list=[]

for idx,line in enumerate(data):
    m = re.search('{', line)
    n = re.search('}',line)
    if m:
        custom_gates_begin.append(idx)
    if n:
        custom_gates_end.append(idx)

# print(len(custom_gates_begin))
number_cust_gates=len(custom_gates_begin)

for i in range(number_cust_gates):
    custom_gates_instructions=[]
    custom_gates_trgts=[]
    for idx,line in enumerate(data):
        if (idx>custom_gates_end[i]-1)or(idx<custom_gates_begin[i]+1):
            continue
        else:
            words=line.split()
            if words:
                custom_gates_instructions.append(words[0])
                custom_gates_trgt=words[1].split(",")
                custom_gates_trgt=[s.replace(';','') for s in custom_gates_trgt]
    #             print(custom_gates_trgt)
                custom_gates_trgts.append(custom_gates_trgt)
    words=data[custom_gates_begin[i]-1].split()
#     print(line)
    targs=words[2].split(',')
    custom_target_dict={}

    j=0
    for i in targs:
        custom_target_dict[i]=j
        j+=1


    custom_gates_trgts_2=[]
    targs=[custom_target_dict[targ] for targ in targs]
    for targ_1 in custom_gates_trgts:

        hold=[]
        for targ_2 in targ_1:
            hold.append(custom_target_dict[targ_2])
        custom_gates_trgts_2.append(hold) 
        
    custom_gates_instructions_list.append(custom_gates_instructions)
    custom_gates_trgts_list.append(custom_gates_trgts_2)


In [ ]:
# seeing as we are trying to do complex conjugate, im going to ignore lines that say measurment or if statements or barrier now...
#measurment is non-unitary... well unless youre into many worlds

In [ ]:
#native gate searches

gate_names=[]
gate_idx=[]
gate_targets=[]


for idx,line in enumerate(data):

    if (idx>custom_idx):
    #get rid of the shit we dont want
        m=re.search('barrier', line)
        n=re.search('measure', line)
        o=re.search('if', line)
        #wont work for more than one custy gate, fix this
        for gate in custom_gates:
            p=re.search(f'{gate}',line)

        if (m==None)and(n==None)and(o==None)and(p==None):

        #get the sh!t we do want. Also formatting...
            words=line.split()
            if words:
#                 print(words)
                gate_names.append(words[0])
                gate_idx.append(idx)
                targets=words[1].split(",")

                targets=[s.replace(';', '') for s in targets]
                targets=[s.replace('[', '_') for s in targets]
                targets=[s.replace(']', '') for s in targets]
                gate_targets.append(targets)
        if(p):
            #get the sh!t we do want
            words=line.split()
            if words:
#                 print(words)
                gate_names.append(words[0])
                gate_idx.append(idx)
                targets=words[1].split(",")
#                 print(targets)
                for qubit_names in q_names:
                    targets=[s.replace(';', '') for s in targets]
                    targets=[s.replace('[', '_') for s in targets]
                    targets=[s.replace(']', '') for s in targets]
                
                for bit_names in c_names:
                    targets=[s.replace(';', '') for s in targets]
                    targets=[s.replace('[', '_') for s in targets]
                    targets=[s.replace(']', '') for s in targets]
#                 print(targets)
                gate_targets.append(targets)

                    


In [ ]:
#a couple of reports for the key lists we will use to build our qcirquit


print(c_names)
print(c_dims)
print(c_idx)

print(q_names)
print(q_dims)
print(q_idx)

print(custom_gates)
print(custom_idx)

print(gate_names)
print(gate_targets)
print(gate_idx)

print(custom_gates_instructions_list)
print(custom_gates_trgts_list)

In [ ]:
#build the qubits

qreg_list=[]
creg_list=[]
for (q_name,q_dim,q_id) in zip(q_names,q_dims,q_idx):
    qreg_list.append(QuantumRegister(int(q_dim), name=q_name))
for (c_name,c_dim,c_id) in zip(c_names,c_dims, c_idx):
    creg_list.append(ClassicalRegister(int(c_dim), name=c_name))
#     print(creg,qreg)

# print(qreg_list)
init_qc=QuantumCircuit(qreg_list[0])
# print(init_qc)
for q in qreg_list:
    try:
        init_qc.add_register((q))
    except:
        continue



In [ ]:
init_qc.draw()

In [ ]:
#seems to be some problem with the 'name' functionality, at least... cant make it work
# can only do some hacky dict stuff to fix this for now...


target_dict={}
q_dims_ints=[int(x) for x in q_dims]
keys=range(sum(q_dims_ints))
values=[]

c=0
j=0

for i in range(sum(q_dims_ints)):
    
    if (j<q_dims_ints[c]):
        val_str=q_names[c]+'_'+str(j)
        j+=1
        values.append(val_str)
        
    else:
        c+=1
        j=0
        val_str=q_names[c]+'_'+str(j)
        j=1
        values.append(val_str)


for i in keys:
    target_dict[i] = values[i]


inv_target_dict = {v: k for k, v in target_dict.items()}



In [ ]:
# build the quantum circuit. There is definately a more beuatiful way to do this


for gate_name,gate_targ,gate_timestep in zip(gate_names,gate_targets,gate_idx):
    if(gate_name.startswith('u')):
        m=re.search('(0.\d+)\D+(0.\d+)\D+(0.\d+)', gate_name)
        if m:
            init_qc.u(float(m.group(1)),float(m.group(2)),float(m.group(3)),inv_target_dict[gate_targ[0]])
    
    if(gate_name=='x'):
        init_qc.x(inv_target_dict[gate_targ[0]])
    elif (gate_name=='y'):
        init_qc.y(inv_target_dict[gate_targ[0]])
    elif (gate_name=='z'):
        init_qc.z(inv_target_dict[gate_targ[0]])
    elif(gate_name=='rx'):
        init_qc.rx(np.pi,inv_target_dict[gate_targ[0]])
    elif (gate_name=='ry'):
        init_qc.ry(np.pi,inv_target_dict[gate_targ[0]])
    elif (gate_name=='rz'):
        init_qc.rz(np.pi,inv_target_dict[gate_targ[0]])
    elif(gate_name=='h'):
        init_qc.h(inv_target_dict[gate_targ[0]])
    elif (gate_name=='s'):
        init_qc.s(inv_target_dict[gate_targ[0]])
    elif (gate_name=='sdg'):
        init_qc.sdg(inv_target_dict[gate_targ[0]])    
    elif (gate_name=='t'):
        init_qc.t(inv_target_dict[gate_targ[0]])
    elif (gate_name=='tdg'):
        init_qc.tdg(inv_target_dict[gate_targ[0]])
    elif (gate_name=='cx'):
        init_qc.cx(inv_target_dict[gate_targ[0]],inv_target_dict[gate_targ[1]])
    elif (gate_name=='ccx'):
        init_qc.ccx(inv_target_dict[gate_targ[0]],inv_target_dict[gate_targ[1]],inv_target_dict[gate_targ[2]])
    
    for i,custom_name in enumerate(custom_gates):
        if(gate_name==f'{custom_name}'):
            for cust_name,cust_targ, in zip(custom_gates_instructions_list[i],custom_gates_trgts_list[i]):
                print(gate_targ,cust_targ,inv_target_dict[gate_targ[0]])
                if(cust_name.startswith('u')):
                    m=re.search('(0.\d+)\D+(0.\d+)\D+(0.\d+)', gate_name)
                    if m:
                        init_qc.u(float(m.group(1)),float(m.group(2)),float(m.group(3)),inv_target_dict[gate_targ[0]])               
                if(cust_name=='x'):
                    init_qc.x(inv_target_dict[gate_targ[0]])
                elif (cust_name=='y'):
                    init_qc.y(inv_target_dict[gate_targ[0]])
                elif (cust_name=='z'):
                    init_qc.z(inv_target_dict[gate_targ[0]])
                elif(cust_name=='rx'):
                    init_qc.rx(np.pi,inv_target_dict[gate_targ[0]])
                elif (cust_name=='ry'):
                    init_qc.ry(np.pi,inv_target_dict[gate_targ[0]])
                elif (cust_name=='rz'):
                    init_qc.rz(np.pi,inv_target_dict[gate_targ[0]])
                elif(cust_name=='h'):
                    init_qc.h(inv_target_dict[gate_targ[0]])
                elif (cust_name=='s'):
                    init_qc.s(inv_target_dict[gate_targ[0]])
                elif (cust_name=='sdg'):
                    init_qc.sdg(inv_target_dict[gate_targ[0]])    
                elif (cust_name=='t'):
                    init_qc.t(inv_target_dict[gate_targ[0]])
                elif (cust_name=='tdg'):
                    init_qc.tdg(inv_target_dict[gate_targ[0]])
                elif (cust_name=='cx'):
                    init_qc.cx(inv_target_dict[gate_targ[0]],inv_target_dict[gate_targ[1]])
                elif (cust_name=='ccx'):
                    init_qc.ccx(inv_target_dict[gate_targ[0]],inv_target_dict[gate_targ[1]],inv_target_dict[gate_targ[2]])





In [ ]:
gates_sheet=("x", "y", "z", "rx", "ry", "rz", "h", "s", "sdg", "t", "tdg", "cx", "ccx", "SWAP", "CSWAP")

In [ ]:
init_qc.draw()

In [ ]:
# easy way to do the required inverse unitary is to to each gate one by one, from the back, and just 
# add implement each gates inverse one at a time - untwizzle!

In [ ]:
U=init_qc.copy()

In [ ]:
#build the dagger

U_dagger=QuantumCircuit(qreg_list[0])
# print(init_qc)
for q in qreg_list:
    try:
        U_dagger.add_register((q))
    except:
        continue

In [ ]:
for inst, qargs, cargs in reversed(U._data):
    U_dagger._append(inst.inverse(), qargs, cargs)

In [ ]:
U_dagger.draw()

In [ ]:
I_crossedfingers=U.compose(U_dagger)

In [ ]:
I_crossedfingers.draw()

In [ ]:
I_crossedfingers.draw()

Heres the check that it all works, using the sim!

In [ ]:
#Changing the simulator 
backend = Aer.get_backend('unitary_simulator')

#The circuit without measurement
circ = I_crossedfingers

#job execution and getting the result as an object
job = execute(circ, backend)
result = job.result()

#get the unitary matrix from the result object
print(result.get_unitary(circ, decimals=3))